# *Keypoints*

En esta lección experimentaremos con la propiedad visual más importante: los *keypoints* o "[puntos de interés](https://en.wikipedia.org/wiki/Interest_point_detection)" y su aplicación al reconocimiento de objetos.

En el capítulo anterior dedicado al flujo óptico hemos visto que las esquinas o *corners* son fácilmente localizables con precisión en imágenes sucesivas. Recuerda el ejemplo [lk_track.py](../code/LK/lk_track.py), en el que se muestran las [trayectorias](../images/demos/tracks.png).
Esta técnica es muy útil para determinadas aplicaciones, pero cuando se cortan los enlaces (por un movimiento brusco de la cámara o porque el punto se sale de la escena) ya no hay forma de continuar la secuencia.

Ahora vamos a dar un paso más: buscaremos zonas características que puedan identificarse en imágenes completamente nuevas aunque se produzcan desplazamientos, giros o cambios de tamaño. En el vídeo siguiente tenemos a la derecha un objeto que queremos reconocer, y a la izquierda la imagen en tiempo real de la webcam. En gris se muestran los "keypoints" detectados y en verde las "correspondencias" (*matching*) encontradas. Si hay "muchas" correspondencias (el número se muestra en el segundo recuadro) podemos suponer que el objeto está en la escena, aunque alguna correspondencia sea errónea.

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/sift/matching2.mp4" style="border:0"></video> 

Para conseguir esto es necesario:

- Detectar zonas "informativas", donde la superficie de imagen sea abrupta, no lineal. No queremos zonas uniformes ni bordes suaves.

- Estas zonas deben tener un tamaño característico, o "escala", que pueda calcularse fácilmente, para conseguir invarianza al tamaño.

- Hay que calcular un vector de características o "descriptor" de cada punto para medir la similitud entre ellos y encontrar correspondencias.

## Bibliotecas y funciones auxiliares

In [ ]:
import numpy             as np
import cv2               as cv
import matplotlib.pyplot as plt

def fig(w,h):
    plt.figure(figsize=(w,h))

def readrgb(file):
    return cv.cvtColor( cv.imread("../images/"+file), cv.COLOR_BGR2RGB) 

def rgb2gray(x):
    return cv.cvtColor(x,cv.COLOR_RGB2GRAY)

def gray2float(x):
    return x.astype(float) / 255

# para ver imágenes monocromas autoescalando el rango
def imshowg(x):
    plt.imshow(x, 'gray')

# para ver imágenes monocromas de float con rango fijo
def imshowf(x):
    plt.imshow(x, 'gray', vmin = 0, vmax=1)

# para ver imágenes con signo
def imshows(x,r=1):
    plt.imshow(x, 'gray', vmin = -r, vmax=r)

## *Hessian*: detector de no linealidad

Las zonas de la imagen cuya superficie de color o nivel de gris es localmente plana no aportan mucha información. Corresponden a regiones uniformes o que cambian suavemente. Una forma de detectar zonas interesantes es buscar segundas derivadas intensas.

Con el gradiente (primeras derivadas) y el Hessiano (segundas derivadas) construimos un modelo cuadrático de la imagen en un pequeño entorno de cada pixel. Si fuera unidimensional y centrado en el origen sería algo como $I(x)\simeq a+bx+\frac{1}{2}cx^2$. En dos dimensiones:

$$p = \begin{bmatrix}x\\y\end{bmatrix} \;\;\;\; \Delta p = p - p_0$$

$$I(p) \simeq I(p_0) + \nabla I\, \Delta p + \frac{1}{2} \Delta p ^ T \, H \, \Delta p $$

$$\nabla I = \begin{bmatrix}\frac{\partial I}{\partial x}&\frac{\partial I}{\partial y}\end{bmatrix}$$


$$H=\begin{bmatrix}\frac{\partial^2 I}{\partial x^2}&\frac{\partial^2 I}{\partial x\partial y}\\
                   \frac{\partial^2 I}{\partial y\partial x}&\frac{\partial^2 I}{\partial y^2}
\end{bmatrix}$$

A continuación se muestran distintos tipos de modelos locales (para versiones muy suavizadas de la imagen original):

<table border=1>
  <tr>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H3.png" width="160px"/></td>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H4.png" width="160px"/></td>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H5.png" width="160px"/></td>
  </tr><tr>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H1.png" width="160px"/></td>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H2.png" width="160px"/></td>
    <td><img src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/quadmod/H6.png" width="160px"/></td>
  </tr>
</table>

En el determinante Hessiano $det(H)$ indica en qué medida es necesario el término cuadrático (no lineal) para modelar localmente la superficie de imagen.

También se podría usar el Laplaciano, pero es menos útil porque su respuesta no permite distinguir bordes intensos de estructuras más localizadas.



Los elementos de $\nabla I$ y $H$ se calculan mediante filtros de derivación.

In [ ]:
def grad(x):
    gx = cv.Sobel(x,-1,1,0)/8
    gy = cv.Sobel(x,-1,0,1)/8
    return gx,gy

def grad2(x):
    gx,gy = grad(x)
    gxx,gxy = grad(gx)
    #gxx = cv.Sobel(x,-1,2,0); gxy = cv.Sobel(x,-1,1,1)
    gyy = cv.Sobel(gy,-1,0,1)/8
    #gyy = cv.Sobel(x,-1,0,2)
    return gx,gy,gxx,gyy,gxy

La imagen siempre se analiza a una cierta escala. Para ello se hará un suavizado gaussiano previo.

In [ ]:
def gaussian(s,x):
    return cv.GaussianBlur(x,(0,0), s)

Cargamos una imagen de prueba.

In [ ]:
#img = readrgb('d2.jpg')[0:500,0:500]
img = readrgb('pano/pano001.jpg')

x = gray2float(rgb2gray(img))
imshowg(x)

In [ ]:
gx,gy,gxx,gyy,gxy = grad2(gaussian(3,x))
h = gxx*gyy-gxy**2

In [ ]:
fig(12,8); plt.suptitle('segundas derivadas')
plt.subplot(2,2,1); imshowg(gxx); plt.title('$\\partial ^2 I/ \\partial x^2$')
plt.subplot(2,2,2); imshowg(gxy); plt.title('$\\partial ^2 I/ \\partial x \\partial y$')
plt.subplot(2,2,4); imshowg(gyy); plt.title('$\\partial ^2 I/ \\partial y^2$');

El signo del determinante indica si la zona es una mancha compacta, más clara u oscura que su entorno, (positivo) o un punto de silla (negativo).

In [ ]:
fig(18,6)
plt.subplot(1,2,1)
imshows(np.maximum(0,h),0.0001); plt.title('blobs')
plt.subplot(1,2,2)
imshows(np.minimum(0,h),0.0001); plt.title('saddlepoints');

Una forma rápida de encontrar los máximos del detector (*non maximum supression*) puede ser la siguiente:

In [ ]:
def nms(x, t = 0.1):
    m = cv.dilate(x, np.ones((5,5),np.uint8))  # filtro de máximo
    h = np.max(m)
    return (x == m) & (x > t*h)

In [ ]:
k = nms(-h)
fig(12,8)
#imshowg(k)
plt.imshow(k, 'gray', interpolation='bicubic');

Lo anterior es una imagen booleana. De ella extraemos las coordenadas de los puntos:

In [ ]:
py,px = np.where(k)

fig(12,8)
plt.imshow(img); #ax = plt.axis();
plt.plot(px,py,'.r'); #plt.axis(ax);

Disgresión: los puntos de hessiano grande tienen la información relevante, son zonas no lineales y con ellas se puede reconstruir una versión aproximada de la imagen, de forma parecida a lo que hicimos con los bordes de Canny.

In [ ]:
k = nms(abs(h),0.05)

mask = cv.dilate(k.astype(np.uint8), np.ones((5,5),np.uint8))
xx = img.copy()
xx[mask==0] = 0,0,0
dst = cv.inpaint(img,1-mask,3,cv.INPAINT_NS);

fig(15,6)
plt.subplot(1,2,1); plt.imshow(xx);
plt.subplot(1,2,2); plt.imshow(dst);

## Detección automática de escala

Los puntos esquina (*corners*) que vimos en el capítulo anterior tienen un comportamiento parecido, pero como veremos a continuación, el enfoque basado en el Hessiano permite asignar un tamaño característico a los puntos de interés. 

La intensidad del detector depende de la escala de análisis.

In [ ]:
disk = gray2float(rgb2gray(readrgb('disk1.jpg')))

imshowg(disk)

### det Hessian

In [ ]:
def hess(x):
    _,_,gxx,gyy,gxy = grad2(x)
    h = gxx*gyy-gxy**2
    return h

In [ ]:
from ipywidgets import interactive

mxs = 50
scales = [1.1**k for k in range(-1,mxs+1)]
hs = np.array([hess(gaussian(s,disk)) for s in scales])

def fun(k=17):
    imshows(hs[k],0.00001)
    plt.title('$\sigma={:.1f}$'.format(scales[k]))

interactive(fun, k=(0,mxs))

Esto sugiere la posibilidad de detectar automáticamente la escala óptima de cada punto (el tamaño de su "área de influencia"). La idea es construir el espacio de escala $(x,y,\sigma)$ de la imagen (una [secuencia de suavizados][1] cada vez mayores) y encontrar los puntos de ese espacio 3D discretizado que son máximos locales (los que tienen una respuesta mayor que sus 9 vecinos).

[1]: https://en.wikipedia.org/wiki/Pyramid_(image_processing)

Es importante normalizar el valor de Hessiano para que tenga respuestas comparables a distintas escalas. Se puede demostrar que la normalización adecuada para el Hessiano es $\sigma^4$ y que se obtendrán máximos locales en objetos redondeados de radio $\sigma \sqrt{2}$. Lo comprobamos con un par de círculos de tamaño conocido.

In [ ]:
def circle(sz=500, rad=50):
    r = np.arange(-sz//2,sz//2)
    x = r.reshape(-1,1)
    y = r.reshape(1,-1)
    img = x**2 + y**2 <= rad**2
    return img.astype(float)

circle1 = circle(500,80)
circle2 = circle(250,30)

fig(12,4)
plt.subplot(1,2,1); plt.imshow(circle1,'gray'); plt.title('radio 80 px')
plt.subplot(1,2,2); plt.imshow(circle2,'gray'); plt.title('radio 30 px');

In [ ]:
hs1 = np.array([s**4*hess(gaussian(s,circle1))[250,250] for s in scales])
s1 = scales[np.argmax(hs1)]
hs2 = np.array([s**4*hess(gaussian(s,circle2))[125,125] for s in scales])
s2 = scales[np.argmax(hs2)]
print('rad: ',np.sqrt(2)*np.array([s1,s2]))
rads = np.sqrt(2)*np.array(scales)
fig(12,4)
plt.subplot(1,2,1)
plt.plot(hs1)
plt.plot(hs2); plt.xlabel('etapa');
plt.subplot(1,2,2)
plt.plot(rads,hs1,'.-')
plt.plot(rads,hs2,'.-'); plt.xlabel('$\sqrt{2}\sigma$');
plt.suptitle('scale-normalized det Hessian');

Los tamaños se detectan muy bien, teniendo en cuenta que la escala está discretizada.

Mediante este procedimiento vamos detectar los círculos de la imagen siguiente:

In [ ]:
circles = cv.resize(readrgb('circles.jpg'),(0,0), fx=0.1, fy=0.1)
circles = 1-np.pad(gray2float(rgb2gray(circles)),100,mode='constant',constant_values=1)

x = circles
imshowg(x)

In [ ]:
hs = np.array([s**4*hess(gaussian(s,x)) for s in scales])

def fun(k):
    imshows(hs[k],r=0.05)
    plt.title('$\sigma={:.1f}$'.format(scales[k]))

interactive(fun, k=(0,mxs))

Mostramos los extremos más intensos:

In [ ]:
import scipy.ndimage

ks = scipy.ndimage.maximum_filter(hs,3)

S,R,C = np.where((ks==hs) & (hs >0.05))
#print(S,R,C)

fig(7,7)
imshowg(x)
ax = plt.gca()
for s,r,c in zip(S,R,C):
        ax.add_patch(plt.Circle((c,r), np.sqrt(2)*scales[s],color='red',fill=False,lw=3));

La detección es excelente. En imágenes naturales no hay círculos perfectos, pero se detectarán "blobs" (manchas más o menos redondeadas) de diferentes tamaños.

### Laplacian

Veamos la respuesta del Laplaciano, que es un detector más simple de no linealidad en la imagen.

In [ ]:
def lap(x):
    _,_,gxx,gyy,_ = grad2(x)
    l = gxx + gyy
    return l

La normalización de Laplaciano es $\sigma^2$.

In [ ]:
Ls = np.array([s**2*lap(gaussian(s,x)) for s in scales])

def fun(k):
    imshows(Ls[k],r=0.5)
    plt.title('$\sigma={:.1f}$'.format(scales[k]))

interactive(fun, k=(0,mxs))

De nuevo, mostramos los extremos más intensos. En este caso nos fijamos en los mínimos, dado que los "blobs" son negros sobre fondo blanco. Si fuera al revés, seleccionaríamos máximos locales. (El determinante Hessiano positivo detecta ambos tipos de blobs, mientras que el negativo detecta los puntos de silla).

In [ ]:
ks = scipy.ndimage.minimum_filter(Ls,3)

S,R,C = np.where((ks==Ls) & (Ls <-0.3))

fig(7,7)
imshowg(x)
ax = plt.gca()
for s,r,c in zip(S,R,C):
        ax.add_patch(plt.Circle((c,r), np.sqrt(2)*scales[s],color='red',fill=False,lw=3));

La detección es igual de precisa y el radio tiene la misma relación con la escala de máxima respuesta.

A continuación se muestran animaciones del Laplaciano para escalas crecientes. Cuando se alcanza el máximo local se marca un círculo con la escala detectada.

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/lapcir2.mp4" style="border:0"></video> 

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/lapjp2.mp4" style="border:0"></video> 

### DoG

Finalmente, vamos a calcular una aproximación al Laplaciano computacionalmente eficiente, basada en la diferencia de niveles consecutivos de suavizado. Es el operador [DoG](https://en.wikipedia.org/wiki/Blob_detection#The_difference_of_Gaussians_approach).

In [ ]:
sigma = 6
deltasigma = 1

s1 = gaussian(sigma,x)
s2 = gaussian(sigma+deltasigma, x)

al = (s2-s1)*sigma/deltasigma
l = lap(gaussian(sigma,x))*sigma**2

fig(10,5)
plt.subplot(1,2,1); plt.imshow(l,'gray'); plt.title('Laplaciano')
plt.subplot(1,2,2); plt.imshow(al,'gray'); plt.title('DoG');

In [ ]:
fig(10,4)
plt.plot(al[275],label='$\\frac{\sigma}{\Delta \sigma}(G_{\sigma + \Delta \sigma}(I) - G_{\sigma}(I)$')
plt.plot(l[275], label='$\sigma^2 \\nabla^2 G_\sigma(I)$');
plt.legend(fontsize=16);

Creamos el espacio de escala. Para que el proceso sea más eficiente habría que ir reduciendo la imagen de tamaño, pero nosotros ahora no lo hacemos para poder comparar las etapas de forma directa.

In [ ]:
mxs = 20
scales = [1.4**k for k in range(-1,mxs+1)]

In [ ]:
pyr = np.array([gaussian(s,x) for s in scales])

In [ ]:
def fun(k=7):
    plt.imshow(pyr[k],'gray',vmin=pyr.min(),vmax=pyr.max());
    plt.title('$\sigma={:.1f}$'.format(scales[k]))

interactive(fun, k=(0,mxs))

Ahora calculamos las diferencias entre ellas. Gracias a que las escalas van aumentando de forma exponencial el factor de ajuste $\sigma/\Delta\sigma$ es constante.

In [ ]:
f = 1/(1.4-1)

dog = np.array([(xa-x)*f for x,xa in zip(pyr,pyr[1:])])

In [ ]:
def fun(k=7):
    plt.imshow(dog[k],'gray',vmin=dog.min(),vmax=dog.max());
    plt.title('$\sigma={:.1f}$'.format(scales[k]))

interactive(fun, k=(0,mxs))

In [ ]:
ks = scipy.ndimage.minimum_filter(dog,3)

In [ ]:
S,R,C = np.where((ks==dog) & (abs(dog) >0.3))
S,R,C

In [ ]:
fig(7,7)
imshowg(x)
ax = plt.gca()
for s,r,c in zip(S,R,C):
        ax.add_patch(plt.Circle((c,r), np.sqrt(2)*scales[s],color='red',fill=False,lw=2));

A medida que $\Delta \sigma$ aumenta la detección de tamaño pierde algo de precisión pero el resultado es muy aceptable.

- [ejemplo](https://www.nature.com/articles/s41598-018-19379-x)

- [automatic scale selection](https://ags.cs.uni-kl.de/fileadmin/inf_ags/opt-ss14/OPT_SS2014_lec03.pdf)

## SIFT: *scale invariant feature transform*

Es uno de los [métodos más conocidos](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform) para extraer puntos de interés. Obtiene puntos característicos con una escala y orientación, y con un *descriptor* de apariencia que permite calcular coincidencias entre puntos de diferentes imágenes. Esto permite resolver muchos problemas de visión artificial, tanto de reconocimiento de objetos como de geometría visual. La detección se basa en máximos locales del determinante Hessiano en el espacio de escala aprovechando el método DoG anterior, y el descriptor es un histograma de gradientes orientados HOG muy parecido al que ya hemos estudiado, tomado en una región de imagen del tamaño indicado por la escala del punto y rotado en la dirección dominante.

OpenCV proporciona una implementación de SIFT. Para calcular a la vez los puntos y los descriptores usamos `detectAndCompute`. Los puntos son estructuras con los campos que se muestran en el siguiente trozo de código, y los descriptores se devuelven como filas de un array. El número de puntos y su intensidad se controla con los argumentos de [SIFT_create](https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html).

In [ ]:
sift = cv.SIFT_create(nfeatures=0, contrastThreshold = 0.04)

In [ ]:
img = readrgb('pano/pano001.jpg')

kp,desc = sift.detectAndCompute(img, mask=None)

# posición
xs = [ p.pt[0] for p in kp]
ys = [ p.pt[1] for p in kp]
# tamaño
sc = [ p.size  for p in kp]
# orientación
rs = [ p.angle for p in kp]

print(len(kp))
print(desc.shape)

In [ ]:
# para mostrar un keypoint en matplotlib, con su tamaño y orientación
def shkeypoint(k):
    t = np.linspace(k.angle,360+k.angle,36)*np.pi/180
    xs = k.pt[0]+k.size*np.cos(t)
    ys = k.pt[1]+k.size*np.sin(t)
    plt.plot(np.append(k.pt[0],xs),np.append(k.pt[1],ys))

fig(12,8)
plt.imshow(img); ax = plt.axis();
for k in kp:
    if k.size > 0:
        shkeypoint(k)
plt.axis(ax);

In [ ]:
img = readrgb('sift/models/vermeer_1.jpg')

kp,desc = sift.detectAndCompute(img, mask=None)

fig(16,16)
plt.imshow(img); ax = plt.axis();
for k in kp:
    if k.size > 10:
        shkeypoint(k)
plt.axis(ax);

Veamos los puntos SIFT de la imagen de círculos anterior. Esta vez usamos la utilidad que proporciona OpenCV para dibujar los puntos.

In [ ]:
circ = rgb2gray(cv.resize(readrgb('circles.jpg'),(0,0), fx=0.1, fy=0.1))
circ = 255-cv.cvtColor(np.pad(circ,100,mode='constant',constant_values=255), cv.COLOR_GRAY2BGR)

kp, desc = sift.detectAndCompute(circ, mask=None)
flag = cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
cv.drawKeypoints(circ,kp,circ, color=(255,0,0), flags=flag)

fig(6,6)
print(len(kp))
plt.imshow(circ,'gray');

Parece que el `.size` calculado se refiere al $\sigma$ óptimo del blob, no al radio del "círculo equivalente".

El vídeo siguiente es una captura del ejemplo de código [`sift.py`](../code/SIFT/sift.py). Cuando aparece la [calculadora](http://mycalcdb.free.fr/main.php?l=0&id=295) podemos observar que los puntos tienen a mantener un tamaño coherente con el del objeto.

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/sift/sift.mp4" style="border:0"></video> 

### Image matching using keypoints

En el ejemplo de código anterior, cuando pulsamos la tecla 'c' guardamos la imagen como modelo y a partir de ahí utilizamos los descriptores de los puntos de interés para intentar encontrar coindicencias:

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/sift/matching1.mp4" style="border:0"></video> 

Los "matching" entre puntos se obtienen cada vez, partiendo de cero, sin realizar ningún tipo de "tracking". El modelo puede desaparecer de la escena y cuando aparece de nuevo se recuperan las asociaciones de puntos. Esto no ocurría así con las trayectorias de "corners" obtenidas mediante el flujo óptico de Lucas-Kanade: cuando se interrumpen ya no pueden asociarse fácilmente en los frames siguientes. El descriptor SIFT utilizado para caracterizar los puntos permite identificarlos en imágenes nuevas de diferentes tamaños y orientaciones, resistiendo también pequeñas inclinaciones.

<video controls src="https://github.com/albertoruiz/umucv/raw/refs/heads/master/images/demos/sift/matching2.mp4" style="border:0"></video> 

Algunas correspondencias son erróneas. Más adelante veremos que hay métodos para eliminarlas. Lo importante es que la proporción de coincidencias respecto al número total de puntos de interés indica la apareción del modelo en la imagen. Este método supuso una revolución en la visión artificial. Hasta aquel momento el ataque más utilizado para reconocer objetos se basaba en una "segmentación" explícita del objeto. Los puntos de interés son una alternativa directa, basada en propiedades estadísticas de la imagen, sin segmentación previa.

Una aplicación directa es el reconocimiento de objetos: la similitud de dos imágenes se puede medir con el número de keypoints coincidentes (que tienen descriptores muy parecidos).

La carpeta sift/models contiene tres imágenes de buena calidad de cuadros de Vermeer. En la carpeta sift/tests hay fragmentos de esas obras tomadas de cualquier manera con la webcam.

In [ ]:
import glob
def readfiles(path):
    return [readrgb(file) for file in sorted(glob.glob('../images/'+path))]

imgs = readfiles('sift/tests/*.*')
mods = readfiles('sift/models/*.*')

In [ ]:
plt.imshow(imgs[1]);

Calculamos los puntos SIFT de todas las imágenes:

In [ ]:
sift = cv.SIFT_create()

dis = [sift.detectAndCompute(x,None) for x in imgs]
dms = [sift.detectAndCompute(x,None) for x in mods]

Las coincidencias de keypoints entre dos imágenes se calculan con un ["matcher"](https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_matcher/py_matcher.html) y luego se filtran para dejar solo las que que no tengan ambigüedad. Una forma de hacer esto es el *ratio test*: se descartan los puntos cuya mejor coincidencia es parecida a la segunda mejor. Para conseguir esta información usamos el método *knn* (k-*nearest neigbors*) del *matcher*.

In [ ]:
# por fuerza bruta
bf = cv.BFMatcher()


# fast approximate nearest neighbor
# pip install opencv-contrib-python

#FLANN_INDEX_KDTREE = 0
#index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
#search_params = dict(checks=50)   # or pass empty dictionary
#flann = cv.FlannBasedMatcher(index_params,search_params)

Cambiando el modelo `m` y la imagen de prueba `t` podemos comprobar que el número de puntos coincidentes o su proporción puede servir para reconocer objetos con suficiente textura. 

In [ ]:
def compare(m,t):
    print('modelo: {},  imagen: {}'.format(len(dms[m][1]), len(dis[t][1])))

    fig(12,8)
    plt.subplot(1,2,1); plt.imshow(mods[m]);
    plt.subplot(1,2,2); plt.imshow(imgs[t]);

    #                     query      model    num de vecinos
    matches = bf.knnMatch(dis[t][1],dms[m][1],k=2)
    #matches = flann.knnMatch(dis[t][1],dms[m][1],k=2)    

    # ratio test
    good = []
    for mt in matches:
        if len(mt) == 2:
            best, second = mt
            if best.distance < 0.75*second.distance:
                good.append(best)        


    print('coincidencias: {} ({:.1f}%)'.format(len(good),100*len(good)/len(dis[t][1])))


    return good

In [ ]:
m = 0
t = 1

good = compare(m,t)

Los *matchings* contienen el grado de coincidencia y los índices de los puntos de interés de la imagen desconocida y del modelo.

In [ ]:
matching = good[0]

matching.distance

In [ ]:
matching.queryIdx

In [ ]:
matching.trainIdx

In [ ]:
m = 0
t = 0

good = compare(m,t)

La función `cv.drawMatches` representa gráficamente las posiciones de los puntos coincidentes.

In [ ]:
fig(12,8)
plt.imshow(cv.drawMatches(imgs[t], dis[t][0], mods[m], dms[m][0], good,
                      flags=0,
                      matchColor=(128,255,128),
                      singlePointColor = (128,128,128),
                      outImg=None) );

### Otros *Keypoints* disponibles en OpenCV

OpenCV tiene muchos otros detectores de *keypoints*, que suelen utilizarse con un interfaz común.

In [ ]:
img = readrgb('pano/pano001.jpg')

#method = cv.xfeatures2d.StarDetector_create()
#method = cv.FastFeatureDetector_create()
#method = cv.ORB_create()
#method = cv.xfeatures2d.SIFT_create()
#method = cv.xfeatures2d.SURF_create()
method = cv.AKAZE_create()

kp = method.detect(img)

xs = [ p.pt[0] for p in kp]
ys = [ p.pt[1] for p in kp]

fig(12,8)
plt.imshow(img); ax = plt.axis('off');
plt.plot(xs,ys,'.r'); plt.axis(ax);

Dependiendo de las necesidades de nuestra aplicación (velocidad de cálculo vs estabilidad de los puntos) elegiremos el más adecuado.

## *Bag of visual words*

Cuando hay muchos modelos la búsqueda de coincidencias en todos ellos puede ser muy costosa. Una forma de acelerar el proceso es construir un "[vocabulario visual](https://en.wikipedia.org/wiki/Bag-of-words_model_in_computer_vision)" a partir de una gran colección de keypoints extraídos de los modelos disponibles o de escenas parecidas (mediante [k-means](https://en.wikipedia.org/wiki/K-means_clustering) o algún método parecido). Cada imagen se representa mediante un histograma que contabiliza el número de ocurrencias de cada "visual word". Este histograma sirve como *feature* para comparar las imágenes o para entrenar un clasificador. ([Csurka 2004](https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/csurka-eccv-04.pdf)).